In [1]:
import numpy as np
import scipy.io
import pandas as pd
from time import time

In [2]:
def split_traindata(data,m,index):
    num=data.shape[0]                  #样本数 
    conum=int(num/m)
    if i<m-1:
        minibatch=data[conum*(index):conum*(index+1),:]
    else:
        minibatch=data[conum*(index):,:]
    return minibatch

In [3]:
def columnkernel(sample,matrix,arg):
    xrow,xcol=matrix.shape
    kernelx=np.zeros(xcol)
    dx=np.zeros(xcol)
    for i in range(xrow):
        dx=(matrix[i,:]-sample)*(matrix[i,:]-sample)/-(2*arg*arg)
        kernelx=kernelx+np.exp(dx)
    return kernelx

In [4]:
def computegradientf(xdata,ylabel,C,w0,arg,theta,tau,ma,sa,it):
    #initialize
    xrow,xcol=xdata.shape
    yrow=ylabel.shape[0]
    b=np.ones(xrow)
    X=np.column_stack((xdata,b))
    margin=np.zeros(xrow)
    W0=np.tile(w0,(xrow,1))
    W=np.zeros((xrow,xcol+1))
    Dtheta=np.tile(theta,(xrow,1))
    taonew=np.zeros(xcol+1)
    wnew=np.zeros(xcol+1)
    beta1=0.9
    beta2=0.999
    aZERO=1e-08
    eta=0.001
    #begin
    margin=b-ylabel@(W0@X.T)
    t=margin>0
    f=margin<=0
    W[t]=C*W0[t]*Dtheta[t]-ylabel[t]@X[t]
    W[f]=C*W0[f]*Dtheta[f]
    a=abs(w0)>tau
    taonew=np.mean(W,axis=0)
    ma[a]= beta1*ma[a] + (1-beta1)*taonew[a]
    sa[a] = beta2*sa[a] + (1-beta2)*(taonew[a]*taonew[a])
    ma[a] = ma[a]/(1-pow(beta1,it))
    sa[a] = sa[a]/(1-pow(beta2,it))
    wnew[a]=w0[a] -((ma[a]*eta)/(np.sqrt(sa[a])+aZERO))
    theta[a]=1/wnew[a]*wnew[a]
    return wnew,theta,ma,sa

In [5]:
def trainl0norm(trainset,C,theta,arg,m,tau):
    #initialize
    epoch=100
    row,col=trainset.shape
    w0=np.ones(col)
    W=np.zeros(col)
    ma=np.zeros(col)
    sa=np.zeros(col)
    #iteration
    for i in range(1,epoch):
        for j in range(m):
            minibatch=split_traindata(trainset,m,j)
            xdata=minibatch[:,:-1]
            ylabel=minibatch[:,-1]
            wnew,theta,ma,sa=computegradientf(xdata,ylabel,C,w0,arg,theta,tau,ma,sa,i)
            w0=wnew
    return w0

In [6]:
def testl0norm(testset,trainset,Wk,arg):
     # computing the size of testset
    col=trainset.shape[1]
    nts=testset.shape[0]
    #generate input X and output y
    Xts=testset[:, 0:col-1]
    ActualY=testset[:, col-1]
    # defining the size of output
    PredictedY=np.zeros(nts)
    for i in range(nts):
        XTS=np.append(Xts[i,:],[1])
        PredictedY[i]=np.sign(Wk.T@XTS)
    return ActualY,PredictedY

In [7]:
# 预处理数据集
data=pd.read_csv('C:\Python310\data\\biary\\MUSK.csv',sep=',')
dataset=data.values[:,2:]
n=dataset.shape[0]
m=dataset.shape[1]
for i in range(n):
    for j in range(m):
        dataset[i,j]=np.float64(dataset[i,j])
mn=dataset.tolist()
dataset=np.array(mn)
for j in range(m-1):
    x_average=np.mean(dataset[:,j])
    s=np.std(dataset[:,j])
    for i in range(n):
        dataset[i,j]=(dataset[i,j]-x_average)/s        
rp = np.random.shuffle(dataset)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Python310\\data\\biary\\MUSK.csv'

In [8]:
#initialize
row,col=dataset.shape
x=5                                # 5折交叉验证也可以是10，20
m=int(row/x)
bestaccu=0
st=time()
arg=1.0
bestaccu=0
bestaver=0
bestvari=0
bestm=0
bestC=0
besttau=0
C2=[0.1]
tau2=[0.1,0.5,0.05]
bestlecord=[]
bestforecast=[]
#iteration
for C in C2:
    for tau in tau2:
        for mini in range(2,10):
            labelrecord=[]
            forecastrecord=[]
            wrecord=np.zeros(col)
            sumpredict=[]
            bestaccu=0
            for i in range(x):
                testset=dataset[i*m:(i+1)*m,:]
                trainset=np.delete(dataset,[i*m,(i+1)*m],axis=0)
                theta=np.ones(col)/(col-1)
                nrow=testset.shape[0]
                Wk=trainl0norm(trainset,C,theta,arg,mini,tau)
                print(Wk)
                ActualY,PredictedY=testl0norm(testset,trainset,Wk,arg)
                accu=0
                for t in range(nrow):
                    labelrecord.append(ActualY[t])
                    forecastrecord.append(PredictedY[t])
                    if ActualY[t]==PredictedY[t]:
                        accu=accu+1
                    et=time()
                print("time:",et-st,accu/nrow)
                sumpredict.append(accu/nrow)
                wrecord=Wk[:]
            aver=np.mean(sumpredict)
            vari=np.var(sumpredict)
            print(len(sumpredict),aver,vari,mini,C,tau)
            if aver>bestaver:
                bestaver=aver
                bestm=mini
                bestvari=vari
                bestC=C
                bestlecord=labelrecord.copy()
                bestforecast=forecastrecord.copy()
                besttau=tau
            print(bestaver,bestvari,bestm,bestC,besttau)

NameError: name 'dataset' is not defined